In [22]:
import pandas as pd
import numpy as np
import ast
import catboost
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
import pickle
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('game_data_final_win_loss.csv')
data.head(5)

,fens,plies,w_elo,b_elo,w_time,b_time,inc_time,stock_evals,result
0,rn3r1k/pp3ppp/4p3/3p1N2/8/2NP2PP/PPP3P1/2KR3R ...,34,2059,1941,833.51,559.17,10.00,"{'e6f5': 239, 'h8g8': 458, 'b8c6': 476, 'h7h6'...",1
1,3r3k/1p3ppp/p1n1r3/3N1p2/1PP5/P2P2PP/3K2P1/4R2...,46,2059,1941,781.44,483.19,10.00,"{'g7g5': 232, 'h8g8': 253, 'b7b5': 254, 'g7g6'...",1
2,4k3/1p6/p5pp/3p1p2/1PP4P/P1K3P1/6P1/8 w - - 0 34,67,2059,1941,702.54,307.39,10.00,"{'c4d5': 206, 'c3d4': 131, 'c3d3': 130, 'a3a4'...",1
3,8/1p6/p2k2pp/3P1p2/PP5P/2K3P1/6P1/8 w - - 1 36,71,2059,1941,684.55,316.30,10.00,"{'c3c4': 284, 'h4h5': 283, 'c3d4': 229, 'g3g4'...",1
4,r2q1rk1/1pp1b1pp/2np4/p7/2Pp4/P4BPP/1P1BQP2/4R...,36,2743,2319,602.32,790.94,0.00,"{'e7f6': 34, 'f8e8': 72, 'e7g5': 82, 'f8f3': 1...",1


In [118]:
idx = 0
samp_eval = ast.literal_eval(data['stock_evals'].values[idx])
print(samp_eval)
move_evals = np.array(list(samp_eval.values()))
loss = np.abs(move_evals - move_evals[0])
is_white = plies[idx] % 2
alpha = (is_white*w_elos[idx] + (1 - is_white)*b_elos[idx])/3300
print(alpha)
expo_loss = np.exp(-4*alpha*0.01*loss)
print(loss)
print(expo_loss/sum(expo_loss))
#print(len(expo_loss))
total_expo_loss = np.sum(expo_loss)
expected_loss = np.dot(expo_loss, loss)/total_expo_loss
print(total_expo_loss)
print(len(loss))

{'e6f5': 239, 'h8g8': 458, 'b8c6': 476, 'h7h6': 477, 'b8a6': 481, 'b7b5': 481, 'f7f6': 489, 'a7a5': 496, 'f8d8': 501, 'f8g8': 502, 'f8e8': 502, 'g7g6': 503, 'a7a6': 506, 'f8c8': 509, 'b7b6': 509, 'b8d7': 512, 'h7h5': 525, 'g7g5': 573, 'e6e5': 624, 'd5d4': 614}
0.5881818181818181
[  0 219 237 238 242 242 250 257 262 263 263 264 267 270 270 273 286 334
 385 375]
[9.59500938e-01 5.55079170e-03 3.63442043e-03 3.54991047e-03
 3.23106981e-03 3.23106981e-03 2.67672789e-03 2.27028234e-03
 2.01832476e-03 1.97139333e-03 1.97139333e-03 1.92555318e-03
 1.79432961e-03 1.67204873e-03 1.67204873e-03 1.55810110e-03
 1.14753171e-03 3.70947106e-04 1.11739310e-04 1.41378586e-04]
1.0422084651584114
20


In [60]:
stock_evals = data['stock_evals'].values
w_elos = data['w_elo'].values
b_elos = data['b_elo'].values
plies = data['plies'].values
w_plus_all = []
b_plus_all = []
equals_all = []
pos_evals = []
expected_losses = []
std_losses = []
thres = 20
f = 4
for i, eval in enumerate(stock_evals):
    dict = ast.literal_eval(eval)
    move_evals = np.array(list(dict.values()))
    pos_evals.append(move_evals[0])
    loss = np.abs(move_evals - move_evals[0])
    is_white = plies[i] % 2
    alpha = (is_white*w_elos[i] + (1 - is_white)*b_elos[i])/3000
    expo_loss = np.exp(-f*alpha*0.01*loss)
    total_expo_loss = np.sum(expo_loss)
    expected_loss = np.dot(expo_loss, loss)/total_expo_loss
    expected_losses.append(expected_loss)
    std_losses.append(np.sqrt(np.dot(expo_loss, loss*loss)/total_expo_loss- expected_loss*expected_loss))
    w_plus = b_plus = equals = 0
    for item in dict.values():
        if item > thres:
            w_plus += 1
        elif item < -thres:
            b_plus += 1
        else:
            equals += 1
    w_plus_all.append(w_plus)
    b_plus_all.append(b_plus)
    equals_all.append(equals)

In [61]:
data_x = data.iloc[:, 1:-2]
data_x['expected_losses'] = expected_losses
data_x['std_losses'] = std_losses
data_x['pos_evals'] = pos_evals
data_x['white_plus'] = w_plus_all
data_x['black_plus'] = b_plus_all
data_x['equals'] = equals_all
data_x.head(5)

,plies,w_elo,b_elo,w_time,b_time,inc_time,expected_losses,std_losses,pos_evals,white_plus,black_plus,equals
0,34,2059,1941,833.51,559.17,10.00,5.72,37.51,239,20,0,0
1,46,2059,1941,781.44,483.19,10.00,42.47,34.43,232,37,0,0
2,67,2059,1941,702.54,307.39,10.00,30.10,43.83,206,8,3,1
3,71,2059,1941,684.55,316.30,10.00,6.09,17.35,284,3,7,1
4,36,2743,2319,602.32,790.94,0.00,17.85,28.57,34,32,0,0


In [62]:
x = data_x.values
y = data['result'].values + 1
# x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size = 0.3, shuffle = True, random_state = 0)
# x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size = 1.0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, shuffle = True, random_state = 0)

BASIC STOCKFISH BASED MODEL

In [66]:
bd = 50 #equality threshold
test_evals = x_test[:, 8]
y_pred_stock = np.array([it > bd for it in test_evals]).astype(int) - np.array([it < -bd for it in test_evals]).astype(int) + 1
#y_true = ohe.inverse_transform(y_test).reshape(-1)
print(f'Accuracy = {accuracy_score(y_test, y_pred_stock)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_stock)*100}%')
print(f'Precision = {precision_score(y_test, y_pred_stock, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_test, y_pred_stock, average = mode)*100}%')
confusion_matrix(y_test, y_pred_stock)

Accuracy = 71.9184468008811%
Balanced Accuracy = 68.8888913314421%
Precision = 74.85438224187483%
F1 Score = 72.9310289797178%


array([[27769,  6954,  4488],
       [ 2619,  9058,  4026],
       [ 2902,  6420, 33369]], dtype=int64)

In [23]:
ohe = OneHotEncoder(sparse_output = False)
y_ohe = ohe.fit_transform(y.reshape(-1, 1))
print(ohe.categories_)

[array([0, 1, 2], dtype=int64)]


In [41]:
del model_nn

In [26]:
x_train_nn, x_test_val_nn, y_train_nn, y_test_val_nn = train_test_split(x, y_ohe, test_size = 0.3, shuffle = True, random_state = 0)
x_val_nn, x_test_nn, y_val_nn, y_test_nn = train_test_split(x_test_val_nn, y_test_val_nn, shuffle = False, test_size = 0.6)

In [42]:
ni = len(x_train[0])
no = len(y_ohe[0])
model_nn = Sequential([
    Dense(256, input_shape = (ni,), activation = 'relu'),
    #Dropout(0.1),
    Dense(256, activation = 'relu'),
    #Dropout(0.1),
    #Dense(400, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(no, activation = 'softmax')
])

In [43]:
model_nn.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [44]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5)
model_nn.fit(x_train_nn, y_train_nn, epochs = 50, validation_data = (x_val_nn, y_val_nn), batch_size = 32, callbacks = [early_stopping_callback])

Epoch 1/50
7117/7117 [==============================] - 21s 3ms/step - loss: 1.3347 - accuracy: 0.7313 - val_loss: 0.6138 - val_accuracy: 0.7612
Epoch 2/50
7117/7117 [==============================] - 21s 3ms/step - loss: 0.6050 - accuracy: 0.7574 - val_loss: 0.6071 - val_accuracy: 0.7357
Epoch 3/50
7117/7117 [==============================] - 24s 3ms/step - loss: 0.5796 - accuracy: 0.7618 - val_loss: 0.5713 - val_accuracy: 0.7699
Epoch 4/50
7117/7117 [==============================] - 22s 3ms/step - loss: 0.5722 - accuracy: 0.7663 - val_loss: 0.5603 - val_accuracy: 0.7738
Epoch 5/50
7117/7117 [==============================] - 20s 3ms/step - loss: 0.5696 - accuracy: 0.7676 - val_loss: 0.5712 - val_accuracy: 0.7728
Epoch 6/50
7117/7117 [==============================] - 21s 3ms/step - loss: 0.5681 - accuracy: 0.7702 - val_loss: 0.5654 - val_accuracy: 0.7624
Epoch 7/50
7117/7117 [==============================] - 21s 3ms/step - loss: 0.5661 - accuracy: 0.7723 - val_loss: 0.5721 - val_ac

In [45]:
mode = 'weighted'
y_pred_nn = model_nn.predict(x_test_nn)
y_pred_nn = np.argmax(y_pred_nn, axis = 1)
y_true_nn = ohe.inverse_transform(y_test_nn).reshape(-1)
print(f'Accuracy = {accuracy_score(y_true_nn, y_pred_nn)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_true_nn, y_pred_nn)*100}%')
print(f'Precision = {precision_score(y_true_nn, y_pred_nn, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_true_nn, y_pred_nn, average = mode)*100}%')
confusion_matrix(y_true_nn, y_pred_nn)

1831/1831 [==============================] - 6s 3ms/step
Accuracy = 79.03795912094668%
Balanced Accuracy = 72.56765094319591%
Precision = 78.35163712628456%
F1 Score = 78.45084774045826%


array([[19785,  1262,  2515],
       [ 2123,  4492,  2801],
       [ 2343,  1232, 22010]], dtype=int64)

In [46]:
model_nn.save('model_nn.keras')
model_nn.save('model_nn.h5')

In [142]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train)
# x_val_scaled = scaler.transform(x_val)
# x_test_scaled = scaler.transform(x_test)

In [ ]:
model_cat = CatBoostClassifier(iterations = 5000, learning_rate = 0.05, depth = 6, eval_metric = 'Accuracy', random_seed = 42, logging_level = 'Verbose', use_best_model = True, early_stopping_rounds = 300)
model_cat.fit(x_train_scaled, y_train, eval_set = (x_val_scaled, y_val))
y_pred_cat = model_cat.predict(x_test_scaled)
print(f'The accuracy of the catboost model is: {accuracy_score(y_test, y_pred_cat)}')

In [136]:
clf = LazyClassifier(verbose = 1, ignore_warnings = False, custom_metric = None)
models, predictions = clf.fit(x_train, x_test, y_train, y_test)
print(predictions)

  3%|██▊                                                                                | 1/29 [00:27<12:37, 27.04s/it]

ROC AUC couldn't be calculated for AdaBoostClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7789730717347131, 'Balanced Accuracy': 0.7229667847417202, 'ROC AUC': None, 'F1 Score': 0.775764997315236, 'Time taken': 27.040196180343628}


  7%|█████▋                                                                             | 2/29 [01:21<19:28, 43.28s/it]

ROC AUC couldn't be calculated for BaggingClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'BaggingClassifier', 'Accuracy': 0.7871181462698291, 'Balanced Accuracy': 0.7338374775598187, 'ROC AUC': None, 'F1 Score': 0.7842639612468562, 'Time taken': 54.65120458602905}


 10%|████████▌                                                                          | 3/29 [01:22<10:16, 23.70s/it]

...

ROC AUC couldn't be calculated for SVC
multi_class must be in ('ovo', 'ovr')
{'Model': 'SVC', 'Accuracy': 0.7912333726072777, 'Balanced Accuracy': 0.7313774535698546, 'ROC AUC': None, 'F1 Score': 0.7865251845019231, 'Time taken': 2278.0826377868652}
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'


 97%|████████████████████████████████████████████████████████████████████████████▎  | 28/29 [2:02:21<06:57, 417.47s/it]

ROC AUC couldn't be calculated for XGBClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'XGBClassifier', 'Accuracy': 0.8010347830541468, 'Balanced Accuracy': 0.7485266373444378, 'ROC AUC': None, 'F1 Score': 0.7980761544871428, 'Time taken': 3.9556941986083984}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003888 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2245
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 12
[LightGBM] [Info] Start training from score -0.911343
[LightGBM] [Info] Start training from score -1.830111
[LightGBM] [Info] Start training from score -0.826403


100%|███████████████████████████████████████████████████████████████████████████████| 29/29 [2:02:24<00:00, 253.25s/it]

ROC AUC couldn't be calculated for LGBMClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'LGBMClassifier', 'Accuracy': 0.7987124976521012, 'Balanced Accuracy': 0.7455804498255884, 'ROC AUC': None, 'F1 Score': 0.7956645138742477, 'Time taken': 3.0260884761810303}
                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
XGBClassifier                      0.80               0.75    None      0.80   
RandomForestClassifier             0.80               0.75    None      0.80   
LGBMClassifier                     0.80               0.75    None      0.80   
BaggingClassifier                  0.79               0.73    None      0.78   
ExtraTreesClassifier               0.79               0.73    None      0.79   
SVC                                0.79               0.73    None      0.79   
AdaBoostClassifier                 0.78               0.72    None      0.78   
Logistic

In [184]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
model_rf.fit(x_train, y_train)
y_pred_rf = model_rf.predict(x_test)
print(f'The accuracy of the random forest model is {accuracy_score(y_test, y_pred_rf)*100}%.')

KeyboardInterrupt: 

In [149]:
!pip install xgboost

In [6]:
import xgboost as xgb
xgb.__version__

'2.0.3'

In [63]:
model_xgb = xgb.XGBClassifier(
    objective = 'multi:softprob',  # Change this based on your specific problem (e.g., 'multi:softprob' for multi-class classification)
    n_estimators = 1000,
    max_depth = 15,
    learning_rate = 0.1,
    use_label_encoder = False  # Set this to False if you're using XGBoost version >= 1.3.0
)
model_xgb.fit(x_train, y_train)
y_pred_xgb = model_xgb.predict(x_test)
print(f'The accuracy of the xgboost model is {accuracy_score(y_test, y_pred_xgb)*100}%.')

The accuracy of the xgboost model is 81.55524819425234%.


ENSEMBLING THE LIGHTGBM AND XGBOOST MODELS

In [16]:
prob_lgbm = model_lgbm_loaded.predict_proba(x_test)

In [17]:
prob_xgb = model_xgb_loaded.predict_proba(x_test)

In [18]:
y_pred_combine = np.argmax((prob_lgbm + prob_xgb)/2, axis = 1)
y_pred_combine

array([2, 0, 1, ..., 0, 2, 2], dtype=int64)

In [40]:
print('For the ensemble model')
print(f'Accuracy = {accuracy_score(y_test, y_pred_combine)*100}%') #best final model. close analysis. combination of lightgbm and xgboost models
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_combine)*100}%')
print(f'Precision = {precision_score(y_test, y_pred_combine, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_test, y_pred_combine, average = mode)*100}%')
confusion_matrix(y_test, y_pred_combine)

For the ensemble model
Accuracy = 81.93125352184826%
Balanced Accuracy = 76.96668233377673%
Precision = 81.54778659006567%
F1 Score = 81.64209162115128%


array([[33619,  1919,  3673],
       [ 2980,  9089,  3634],
       [ 3258,  2172, 37261]], dtype=int64)

In [219]:
prob_lgbm_train = model_lgbm.predict_proba(x_train)
prob_xgb_train = model_xgb.predict_proba(x_train)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [226]:
x_new_train = np.concatenate((prob_lgbm_train, prob_xgb_train), axis = 1)

In [231]:
x_comb_train, x_comb_val, y_comb_train, y_comb_val = train_test_split(x_new_train, y_train, test_size = 0.3, random_state = 42)

In [ ]:
model_cat = CatBoostClassifier(iterations = 5000, learning_rate = 0.05, depth = 6, eval_metric = 'Accuracy', random_seed = 42, logging_level = 'Verbose', use_best_model = True, early_stopping_rounds = 300)
model_cat.fit(x_comb_train, y_comb_train, eval_set = (x_comb_val, y_comb_val))
y_pred_comb_cat = model_cat.predict(np.concatenate((prob_lgbm, prob_xgb), axis = 1))
accuracy_score(y_test, y_pred_comb_cat)*100

In [227]:
x_new_train.shape

(227743, 6)

In [229]:
model_rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
model_rf.fit(x_new_train, y_train)

RandomForestClassifier(random_state=42)

In [230]:
y_pred_rf_combine = model_rf.predict(np.concatenate((prob_lgbm, prob_xgb), axis = 1))
accuracy_score(y_test, y_pred_rf_combine)*100

81.74683673992111

In [216]:
# Save the model to a file
with open('model_lgbm.pkl', 'wb') as f:
    pickle.dump(model_lgbm, f)

array([2, 0, 1, ..., 0, 2, 2], dtype=int64)

In [14]:
# Load the model from the file
mode = 'weighted'
with open('model_lgbm.pkl', 'rb') as f:
    model_lgbm_loaded = pickle.load(f)
y_pred_lgbm = model_lgbm_loaded.predict(x_test)
print(f'Accuracy = {accuracy_score(y_test, y_pred_lgbm)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_lgbm)*100}%')
print(f'Precision = {precision_score(y_test, y_pred_lgbm, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_test, y_pred_lgbm, average = mode)*100}%')

Accuracy = 81.90973823062343%
Balanced Accuracy = 76.85209122219896%
Precision = 81.51138567985998%
F1 Score = 81.60084976226665%


In [217]:
# Save the model to a file
with open('model_xgb.pkl', 'wb') as f:
    pickle.dump(model_xgb, f)

array([2, 0, 1, ..., 0, 2, 2], dtype=int64)

In [13]:
mode = 'weighted'
with open('model_xgb.pkl', 'rb') as f:
    model_xgb_loaded = pickle.load(f)
y_pred_xgb = model_xgb_loaded.predict(x_test)
print(f'Accuracy = {accuracy_score(y_test, y_pred_xgb)*100}%')
print(f'Balanced Accuracy = {balanced_accuracy_score(y_test, y_pred_xgb)*100}%')
print(f'Precision = {precision_score(y_test, y_pred_xgb, average = mode)*100}%')
print(f'F1 Score = {f1_score(y_test, y_pred_xgb, average = mode)*100}%')

Accuracy = 81.57471441012243%
Accuracy = 76.65638719500657%
Precision = 81.19682328474047%
F1 Score = 81.30774289585752%


In [171]:
!pip install lightgbm

In [64]:
del model_lgbm

NameError: name 'model_lgbm' is not defined

In [65]:
import lightgbm as lgb
model_lgbm = lgb.LGBMClassifier(
    boosting_type = 'gbdt',
    objective = 'multiclass',  # Change this based on your specific problem (e.g., 'multiclass' for multi-class classification)
    num_leaves = 1000,
    learning_rate = 0.1,
    n_estimators = 1000
)
model_lgbm.fit(x_train, y_train)
y_pred_lgbm = model_lgbm.predict(x_test)
print(f'The accuracy of the LGBM model is {accuracy_score(y_test, y_pred_lgbm)*100}%.')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010145 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2236
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 12
[LightGBM] [Info] Start training from score -0.911343
[LightGBM] [Info] Start training from score -1.830111
[LightGBM] [Info] Start training from score -0.826403
The accuracy of the LGBM model is 81.88924747707597%.


In [211]:
del model_bag

In [187]:
from sklearn.ensemble import BaggingClassifier
base_estimator = lgb.LGBMClassifier(
    boosting_type = 'gbdt',
    objective = 'multiclass',  # Change this based on your specific problem (e.g., 'multiclass' for multi-class classification)
    num_leaves = 50,
    learning_rate = 0.05,
    n_estimators = 100
)
model_bag = BaggingClassifier(
    base_estimator=base_estimator,
    n_estimators=50,       # Number of base estimators to train
    max_samples=0.8,       # Fraction of samples to draw from X to train each base estimator
    max_features=1.0,      # Fraction of features to draw from X to train each base estimator
    bootstrap=True,        # Whether samples are drawn with replacement
    bootstrap_features=False,  # Whether features are drawn with replacement
    random_state=42
)
model_bag.fit(x_train, y_train)
y_pred_bag = model_bag.predict(x_test)
print(f'The accuracy of the bagging LGBM model is {accuracy_score(y_test, y_pred_bag)*100}%.')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010788 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2241
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 12
[LightGBM] [Info] Start training from score -0.918318
[LightGBM] [Info] Start training from score -1.822684
[LightGBM] [Info] Start training from score -0.822758
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003462 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2238
[LightGBM] [Info] Number of data points in the train set: 227743, number of used features: 12
[LightGBM] [Info] Start training from score -0.915353
[LightGBM] [Info] Start training from score -1.829569
[LightGBM] [Info] Start training from score -0.822933
[LightGBM] [Info] Auto-choosing 

In [169]:
confusion_matrix(y_test, y_pred_xgb)

array([[20136,  1228,  2259],
       [ 1824,  5464,  2200],
       [ 1957,  1367, 22128]], dtype=int64)

In [97]:
model_cat.save_model('cat1')

In [98]:
model_loaded = CatBoostClassifier()
model_loaded.load_model('cat1')

In [99]:
model_loaded.predict(x_test)

array([[-1],
       [-1],
       [-1],
       ...,
       [-1],
       [ 1],
       [ 1]], dtype=int64)